In [540]:


import numpy as np
import pandas as pd

import sklearn
from sklearn.decomposition import TruncatedSVD

columns = ['id','id2','task','TaskId','rating']
frame = pd.read_csv('DataInterviewVisitor.csv',sep=',', names=columns)
frame.head()

,id,id2,task,TaskId,rating
0,0,1,11010,1,7
1,1,1,10010,2,7
2,2,1,11010,3,8
3,3,1,1000,4,7
4,4,1,10000,5,6


In [541]:
columns2 =['TaskId','TaskName','Task']
tasks = pd.read_csv('TasksInterviewVisitor.csv', sep=',', names = columns2, encoding='latin-1')
task_names = tasks[['TaskId','TaskName','Task']]
task_names.head()


,TaskId,TaskName,Task
0,1,Task1,11010
1,2,Task2,10010
2,3,Task3,11010
3,4,Task4,1000
4,5,Task5,10000


In [542]:
combined_task_data = pd.merge(frame, task_names, on='TaskId')
combined_task_data.head()

,id,id2,task,TaskId,rating,TaskName,Task
0,0,1,11010,1,7,Task1,11010
1,6,1,11010,1,9,Task1,11010
2,7,1,11010,1,7,Task1,11010
3,19,1,11010,1,10,Task1,11010
4,22,1,11010,1,4,Task1,11010


In [543]:
combined_task_data.groupby('TaskId')['rating'].count().sort_values(ascending=True).head()

filter_interview = combined_task_data['TaskId']==1
combined_task_data[filter_interview]['TaskName'].unique()

array(['Task1'], dtype=object)

In [544]:
rating_crosstab = combined_task_data.pivot_table(values='rating', index='id', columns=['TaskName'], fill_value=0)
rating_crosstab

TaskName,Task1,Task10,Task11,Task2,Task3,Task4,Task5,Task6,Task7,Task8,Task9
id,,,,,,,,,,,
0,7,0,0,0,0,0,0,0,0,0,0
1,0,0,0,7,0,0,0,0,0,0,0
2,0,0,0,0,8,0,0,0,0,0,0
3,0,0,0,0,0,7,0,0,0,0,0
4,0,0,0,0,0,0,6,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
96,0,0,0,0,0,0,0,0,9,0,0
97,0,0,0,0,0,0,0,0,9,0,0
98,0,0,0,6,0,0,0,0,0,0,0


In [545]:
rating_crosstab.shape
X = rating_crosstab.T
X.shape

(11, 101)

In [546]:
SVD = TruncatedSVD(n_components=11, random_state=0)

resultant_matrix = SVD.fit_transform(X)

resultant_matrix.shape

(11, 11)

In [547]:
corr_mat = np.corrcoef(resultant_matrix)
corr_mat.shape
task_names = rating_crosstab.columns
task_list = list(task_names)
task_list
task = task_list.index('Task1')
task
corr_star_wars = corr_mat[task]
corr_star_wars.shape
corr_star_wars

array([ 1. , -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1])

In [548]:
list(task_names[(corr_star_wars<1) & (corr_star_wars > -0.1)])


['Task2', 'Task4', 'Task9']